In [ ]:
#Written by Andrew Hamilton, TLU Physics, Class of 2019
#Part of a research project done in collaboration with Dr. Josh Fuchs during the Summer of 2018 through the Fall of 2018

In [ ]:
import numpy as np
import csv
from astropy.table import Table
from astropy.io import ascii

# Functions

## •Organizing Data

In [ ]:
def super_tableizer(data):
    """Super Tableizer
    
    Will combine given tables of astro data (as long as the first 2 columns are ra and dec), into one large table
    
    Args:
        Data (list): List containing data tables that you want to combine together
        
    Return:
        (Astropy.Table): Combined Table
    """
    count = 0
    datatable = Table()
    for dataset in data:
        if count == 0:
            for name in dataset.colnames:
                datatable[name] = dataset[name]
        else:
            for name in dataset.colnames[2:]:
                datatable[name] = dataset[name]
        count += 1
                
    return datatable

In [ ]:
def min_to_deg(value):
    """Min-to-Deg
    Converts RA/Dec values from deg-min-sec format to decimal degrees, to interface with other data formats
    This function should be valid for any dataset using this format, so here's hoping!
    
    Args:
        value (str): dg-mn-se.conds format coordinates
        
    Returns:
        (float): decimal degree coordinates    
    """
    
    degrees = []
    minutes = []
    seconds = []
    
    coord = [degrees, minutes, seconds]
    index_num = 0
    
    for char in value:
        if char == ' ':
            index_num += 1
        else:
            coord[index_num].append(char)
    
    #I didn't know how to turn this set of lists and ints and such to values
    #until I found this solution on ~the Internet~
    #https://stackoverflow.com/a/490020        
    deg_val = int(''.join(map(str,degrees)))
    min_val = int(''.join(map(str,minutes)))
    sec_val = float(''.join(map(str,seconds)))
    
    decimal_coord = deg_val + min_val/60 + sec_val/60**2
            
    return decimal_coord

## •Analyzing Data

In [ ]:
def index_range_finder(color_index, deviations):
    """Color Index Range Finder
    
    Determines the approximate range of a cluster of Color Indices for a LARP
    
    Modelled from a cluster found in the I-Z indices of 8 of the 9 known LARPs
    
    Args:
        color_index (list): contains indices of the known LARPs
        deviations (float?): number of standard deviations from mean to calc range (int or float preferred?)
        
    Returns:
        (float): lower bound of the cluster range
        (float): upper bound of the cluster range
    
    """
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #if you want modular stdevs, here's the place to do so
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    stdev = np.std(color_index)
    low_bound = np.mean(color_index)-stdev*deviations
    up_bound = np.mean(color_index)+stdev*deviations
    
    return low_bound, up_bound

In [ ]:
def auto_cluster_finder(data, a, b, deviations, threshold):
    """Auto Cluster Finder
    
    Automatically finds potential clusters
    
    Args:
        data (astropy.Table): Astropy data containing however many magnitudes
        a (str): First magnitude, will be subtracted from to get index
        b (str): Second magnitude, will be subtracted from a to get index
        deviations (float?): number of standard deviations from mean to calc range (int or float preferred?)
        threshold (float): fraction of stars contained within cluster region for cluster to be considered valid
        
    Returns:
        (list): List containing lbound, ubound, mag_a, mag_b, and if the filter is valid
    """
    
    x = data[a] - data[b]
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #if you want modular stdevs, here's the place to do so
    #go thru the following function to find it!!!!!!!!!!!!
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    c, d = index_range_finder(x, deviations)
    count = 0
    for item in x:
        if c <= item and item <= d:
            count += 1 
            
    percent = count/len(data['ra'])
    
    if percent > threshold:
        print(a + '-' + b)
        print('Suitable Filter found!',str(count),'stars in range.')
        print('Lbound:',str(c),', Ubound:',str(d))
        good_filter = True
    else:
        #uncomment the line below if you want it to print for ~every single possible filter~
        #print('Filter unsuitable. Only',str(count),'stars in range.')
        good_filter = False
        
    return [c, d, a, b, good_filter]

In [ ]:
def filter_generator(data, deviations,threshold):
    """Filter Generator
    
    Generates possible filters for LARPs
    
    Args:
        data (astropy.Table): Astropy data
        deviations (float?): number of standard deviations from mean to calc range (int or float preferred?)
        threshold (float): fraction of stars contained within cluster region for cluster to be considered valid
        
    Returns:
        (list): list containing filters using my standard 1-D filter format, includes insufficient ones
    """
    column_names = data.colnames
    
    x = len(column_names)
    indices = []
    for a in range(2,x):
        for b in range(2,x):
            if a < b:
                #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                #if you want modular stdevs, here's the place to do so
                #go thru the following function to find it!!!!!!!!!!!!
                #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                filt = auto_cluster_finder(data, column_names[a], column_names[b], deviations,threshold)
                if filt[4]:
                    indices.append(filt)
    return indices

In [ ]:
def compare_indices(filt, row):
    """Compare Indices
    
    Determines if an index falls into the filter range
    
    Args:
        filt (list): filter information (in standard 1-D filter format)
        row (astropy.Table): magnitude data on one star
        
    Returns:
        (boolean): Whether or not the star passes all the filter conditions
    """
    #allow me to detail the 1-D filter format
    
    #filter = [lower bound, upper bound, magnitude A, magnitude B, Boolean if the filter is valid]
    #filter = [lbound, ubound, a, b, True/False]
  
    
    a = filt[2]
    b = filt[3]
    
    if a not in row.colnames or b not in row.colnames:
        return False
    else:
        x = float(row[a]) - float(row[b])

        if filt[0] <= x and filt[1] >= x:
            return True
        else:
            return False

In [ ]:
def index_filterator(data, deviations = 1, threshold = 2/3):
    """Index Filterator
    
    Stores coordinates of stars that pass filter conditions in table, can be written to file
    
    Args:
        data (astropy.Table): Astropy data
        deviations (float?): number of standard deviations from mean to calc range (int or float preferred?)
        threshold (float): fraction of stars contained within cluster region for cluster to be considered valid
        
    Returns:
        (astropy.Table): Coordinates of LARP candidates
    """
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #if you want modular stdevs, here's the place to do so
    #go thru the following function to find it!!!!!!!!!!!!
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    indices = filter_generator(larp_table, deviations,threshold)
    names_cheat = larp_table.colnames
       
    possible_larps = Table(names=names_cheat)
    
    for row in data:
        passes_test = []
        for index in indices:
            passes_test.append(compare_indices(index, row))
            
        if False not in passes_test:
            possible_larps.add_row(row)
    
    n_larp_cands = len(possible_larps['ra'])
    print('\nThere are',str(len(indices)),'usable filters!')
    print('There are',str(n_larp_cands),'potential LARPs in this dataset!')
    
    return possible_larps

# Data

In [ ]:
#read in data tables
larps_sdss = ascii.read('LARP Characteristics-SDSS.csv')
larps_wise = ascii.read('LARP Characteristics-WISE.csv')
larps_xmm  = ascii.read('LARP Characteristics-XMM.csv')

sdss_table = ascii.read('Skyserver_SQL6_25_2018 7_25_30 PM.csv')
wise_table = ascii.read('wise_results.tbl')
xmm_full = ascii.read('BrowseTargets.8264.1539051979', delimiter='|', header_start=2, data_start=3)

In [ ]:
#Organize LARPs
larp_data = [larps_sdss, larps_wise,larps_xmm]
larp_table = super_tableizer(larp_data)

In [ ]:
#Organize Survey Data
survey_data = [sdss_table, wise_table]
survey_table = super_tableizer(survey_data)

In [ ]:
prelim_larps = index_filterator(survey_table,2,7/9)
#if the threshold (3rd argument) is >=7/9 then it breaks, saying "ValueError: Mismatch between number of vals and columns"
#Figure out what this means and FIX IT PLEASE

# Prelim. Results/Discussion

stdev = 1: 5 Filters, 67 Candidates
stdev = 2: 27 Filters, 0 Candidates

I like hte second one's results, but the 27/36 possible filters being successful gives me heartburn
That's ~kinda~ sus to me, especially given all the "7 stars in range" I'm seeing :thinking:
Would it be worth it to change the amount of LARPs that make a valid filter?

In Mk2, it seems that the lack of masking caused the number of filters to rocket up, but to also shrink the number of potential LARPs... Interesting, but also concerning.

However, with stdev=2, the filters increase, like expected... but to include all possible filters! Worse yet, the number of candidates **INCREASED** I am confident that the issue lies in the values I masked

I believe the issue lied in how the values were recorded in the data. I manually changed the masked values to blank spots. I cannot remember if these were originally zero or blank... I hope it was the latter!